In [1]:
from datetime import date, timedelta
import os
import pickle
import sys
import pandas as pd
import numpy as np
sys.path.append("..")

from abautomator.visualizer import BasicVisualizer, AbsDiffVisualizer, RelDiffVisualizer

from bokeh.io import output_notebook
from bokeh.plotting import show
from bokeh.models import MultiChoice, CustomJS
from bokeh.layouts import column

output_notebook()

Loading BokehJS ...

In [2]:
# %pip install -U pandas

In [3]:
# name = "rollback_analy"
name = "inspiration_analy"
analy = pickle.load(
    open(os.path.join("..", "tests", "cache", f"{name}.p"), "rb" )
)

In [4]:
df = analy.get_rel_diff_confidence_intervals()
df.head()

,exp_cond,metric,tx_mean,tx_std,tx_count,ctrl_mean,ctrl_std,ctrl_count,abs_mean,abs_std,p_value,upper_68_ci,lower_68_ci,upper_95_ci,lower_95_ci,mean,std
0,Video01,n_entered_phone,0.508623,0.500202,37053.0,0.510671,0.500110,36828.0,-0.002048,0.003680,0.577794,0.319531,-1.121787,1.011364,-1.813620,-0.401128,0.720659
1,Video01,pct_entered_phone,0.508488,0.499935,37053.0,0.510563,0.499895,36828.0,-0.002075,0.001839,0.129640,-0.046143,-0.766597,0.299674,-1.112414,-0.406370,0.360227
2,Carousel01,n_entered_phone,0.510820,0.500162,36784.0,0.510671,0.500110,36828.0,0.000149,0.003687,0.967829,0.751059,-0.692825,1.444124,-1.385889,0.029117,0.721942
3,Carousel01,pct_entered_phone,0.510684,0.499893,36784.0,0.510563,0.499895,36828.0,0.000121,0.001842,0.473737,0.384642,-0.337095,0.731075,-0.683528,0.023773,0.360868
4,Carousel02,n_entered_phone,0.507323,0.500333,36801.0,0.510671,0.500110,36828.0,-0.003348,0.003687,0.363838,0.066364,-1.377602,0.759467,-2.070705,-0.655619,0.721983


In [26]:
from abautomator.visualizer import utils, btn
source = utils.convert_df_to_source(df)

In [27]:
OPTIONS = list(df.exp_cond.unique())

vis = RelDiffVisualizer(source)
fig = vis.get_figure()

OPTIONS = list(df.exp_cond.unique())

multi_choice = MultiChoice(value=OPTIONS, options=OPTIONS)
callback = CustomJS(
    args=dict(yRange=fig.y_range, source=source),
    code="""
    console.log('multi_choice: value=' + this.value, this.toString())
    console.log(yRange)
    for (const item of this.value){
        console.log(item);
    }
    const multiChoiceArray = this
    let tempList = source.data["factor_label"].filter(function (currentElement) {
      return multiChoiceArray.value.includes(currentElement[1]);
    });
    for (const item of tempList){
        console.log(item);
    }
    yRange.factors = tempList
    """
)
multi_choice.js_on_change("value", callback)

show(column(multi_choice, fig, btn.get_stat_sig_btn(fig.renderers)))

In [23]:
fig.y_range = FactorRange(*[item for item in source.data["factor_label"] if item[1] != 'Carousel01'])

In [24]:
show(fig)

In [19]:
from bokeh.models import FactorRange
FactorRange(*list(source.data["factor_label"]))

FactorRange(id='1345', ...)

In [22]:
[item for item in source.data["factor_label"] if item[1] != 'Carousel01']

[('N Entered Phone', 'Video01'),
 ('% Entered Phone', 'Video01'),
 ('N Entered Phone', 'Carousel02'),
 ('% Entered Phone', 'Carousel02'),
 ('N Entered Phone', 'Carousel03'),
 ('% Entered Phone', 'Carousel03'),
 ('N Entered Phone', 'Carousel04'),
 ('% Entered Phone', 'Carousel04'),
 ('N Granted Contacts', 'Video01'),
 ('% Granted Contacts', 'Video01'),
 ('N Granted Contacts', 'Carousel02'),
 ('% Granted Contacts', 'Carousel02'),
 ('N Granted Contacts', 'Carousel03'),
 ('% Granted Contacts', 'Carousel03'),
 ('N Granted Contacts', 'Carousel04'),
 ('% Granted Contacts', 'Carousel04')]

In [18]:
from abautomator.visualizer import utils
source = utils.convert_df_to_source(vis.df)

In [5]:
class TestVisualizer(RelDiffVisualizer):
    """ Parent object. Not to be initiated directly """

    def __init__(self, df, x_axis_label):
        self.df = df
        self.x_axis_label = x_axis_label

    def get_figure(self):
        df = self.df.copy()
        df = _clean_exp_conds(df)
        source = ColumnDataSource(df)

        fig = _init_figure(source, self._get_tool_tips())

        _add_zero_span(fig)

        _add_bars(fig, source)

        # _set_legend(fig)
        _set_x_axis(fig, self.x_axis_label)
        _set_y_axis(fig)
        
        return fig

In [6]:
from abautomator.visualizer.parent import _cond_to_color_mapper, add_error_bars, add_core_interval

def _add_sig_bars(fig, source):

    for exp_cond, color in _cond_to_color_mapper(source).items():
        bools = [
            True if p_value <= 0.05 else False
            for p_value in source.data['p_value']
        ]
        view = CDSView(source=source, filters=[BooleanFilter(bools)])

        fig_core = (fig, source, view)

#         lower_eb, upper_eb = add_error_bars(fig_core)
        core_interval = add_core_interval(fig, exp_cond, "blue", fig_core, add_legend=False)
        core_interval.tags = ["stat_sig"]
        core_interval.visible = False

#         core_interval.js_link('visible', lower_eb, 'visible')
#         core_interval.js_link('visible', upper_eb, 'visible')

ImportError: cannot import name '_cond_to_color_mapper' from 'abautomator.visualizer.parent' (../abautomator/visualizer/parent.py)

In [7]:
vis = RelDiffVisualizer(analy.get_rel_diff_confidence_intervals())
fig = vis.get_figure()

from bokeh.layouts import column
from bokeh.models import Button, CustomJS, ColumnDataSource, CDSView, BooleanFilter

button = Button(label="Toggle Stat Sig", button_type="danger")
callback = CustomJS(
    args=dict(
        glyphs=fig.renderers,
        stat_sig=stat_sig_visible
    ),
    code="""
    for (const glyph of glyphs){
        if (glyph.tags.includes("stat_sig")) {
            var stat_sig_visible = glyph.visible;
        }
    }
    console.log("SIG");
    console.log(stat_sig_visible);
    console.log("SIG");
    
    if (stat_sig_visible) {
        var muted = false;
    } else {
        var muted = true;
    }

    for (const glyph of glyphs){
        if (glyph.tags.includes("stat_sig")) {
            glyph.visible = !glyph.visible;
        } else {
            glyph.muted = muted;
        }
    }
    """
    )

button.js_on_click(callback)

# fig.legend.click_policy="mute"
# fig.add_layout(fig.legend[0], 'right')
# fig = _add_sig_bars(fig, ColumnDataSource(analy.get_rel_diff_confidence_intervals()))

show(column(fig, button))
show(fig)

AttributeError: 'Column' object has no attribute 'renderers'

In [16]:
fig.renderers[2]._property_values

{'glyph': HBar(id='2464', ...),
 'nonselection_glyph': HBar(id='2465', ...),
 'view': CDSView(id='2454', ...),
 'data_source': ColumnDataSource(id='2425', ...),
 'js_property_callbacks': {'change:visible': [CustomJS(id='2467', ...),
   CustomJS(id='2468', ...)]},
 'visible': False,
 'tags': ['stat_sig']}

In [8]:
vis = BasicVisualizer(analy.get_basic_confidence_intervals())
fig = vis.get_figure()
# fig.add_layout(fig.legend[1], 'right')
show(fig)

In [6]:
vis = AbsDiffVisualizer(analy.get_abs_diff_confidence_intervals())
show(vis.get_figure())

Carousel01 [False, False, False, False, False, False, True, True, False, False, False, False, False, False, False, False, True, True, False, False]
Carousel02 [False, False, True, True, False, False, False, False, False, False, False, False, True, True, False, False, False, False, False, False]
Carousel03 [False, False, False, False, True, True, False, False, False, False, False, False, False, False, True, True, False, False, False, False]
Carousel04 [False, False, False, False, False, False, False, False, True, True, False, False, False, False, False, False, False, False, True, True]
Video01 [True, True, False, False, False, False, False, False, False, False, True, True, False, False, False, False, False, False, False, False]


Carousel01 [False, False, False, False, False, False, True, True, False, False, False, False, False, False, False, False, True, True, False, False]
Carousel02 [False, False, True, True, False, False, False, False, False, False, False, False, True, True, False, False, False, False, False, False]
Carousel03 [False, False, False, False, True, True, False, False, False, False, False, False, False, False, True, True, False, False, False, False]
Carousel04 [False, False, False, False, False, False, False, False, True, True, False, False, False, False, False, False, False, False, True, True]
Video01 [True, True, False, False, False, False, False, False, False, False, True, True, False, False, False, False, False, False, False, False]


In [9]:
analy.get_rel_diff_confidence_intervals()

,exp_cond,metric,tx_mean,tx_std,tx_count,ctrl_mean,ctrl_std,ctrl_count,abs_mean,abs_std,upper_68_ci,lower_68_ci,upper_95_ci,lower_95_ci,mean,std
0,Video01,n_entered_phone,0.508623,0.500202,37053.0,0.510671,0.500110,36828.0,-0.002048,0.003680,0.319531,-1.121787,1.040191,-1.842447,-0.401128,0.720659
1,Video01,pct_entered_phone,0.508488,0.499935,37053.0,0.510563,0.499895,36828.0,-0.002075,0.001839,-0.046143,-0.766597,0.314083,-1.126823,-0.406370,0.360227
2,Carousel02,n_entered_phone,0.507323,0.500333,36801.0,0.510671,0.500110,36828.0,-0.003348,0.003687,0.066364,-1.377602,0.788346,-2.099584,-0.655619,0.721983
3,Carousel02,pct_entered_phone,0.507133,0.499956,36801.0,0.510563,0.499895,36828.0,-0.003430,0.001842,-0.310889,-1.032592,0.049962,-1.393444,-0.671741,0.360852
4,Carousel03,n_entered_phone,0.512655,0.500392,36666.0,0.510671,0.500110,36828.0,0.001984,0.003691,1.111109,-0.334269,1.833798,-1.056959,0.388420,0.722689
5,Carousel03,pct_entered_phone,0.512382,0.499853,36666.0,0.510563,0.499895,36828.0,0.001819,0.001844,0.717502,-0.004787,1.078646,-0.365931,0.356357,0.361144
6,Carousel01,n_entered_phone,0.510820,0.500162,36784.0,0.510671,0.500110,36828.0,0.000149,0.003687,0.751059,-0.692825,1.473001,-1.414767,0.029117,0.721942
7,Carousel01,pct_entered_phone,0.510684,0.499893,36784.0,0.510563,0.499895,36828.0,0.000121,0.001842,0.384642,-0.337095,0.745510,-0.697963,0.023773,0.360868
8,Carousel04,n_entered_phone,0.506039,0.500188,36681.0,0.510671,0.500110,36828.0,-0.004633,0.003689,-0.184706,-1.629643,0.537762,-2.352111,-0.907175,0.722468
9,Carousel04,pct_entered_phone,0.505930,0.499972,36681.0,0.510563,0.499895,36828.0,-0.004633,0.001844,-0.546300,-1.268606,-0.185146,-1.629759,-0.907453,0.361153


In [8]:
# df["t_calc"] = df["mean"]/df["std"]
# df["t_crit"] = scipy.stats.t.ppf(q=1-.05/2, df=df["tx_count"] + df["ctrl_count"] - 2)
# df["p_value"] = scipy.stats.t.sf(np.abs(df["t_calc"]), df["tx_count"] + df["ctrl_count"] - 2)*2  # two-sided pvalue = Prob(abs(t)>tt)
# df.head()